In [1]:
!pip install --quiet spacy
!python -m spacy download en_core_web_sm


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 92.3 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [2]:
import spacy
import pandas as pd
import numpy as np
import re
from spacy.matcher import PhraseMatcher
from spacy.tokens import Span


In [3]:
DATA_PATH = "/content/synthetic_bank_tweets_1500_with_names.csv"
df = pd.read_csv(DATA_PATH).reset_index(drop=True)
df['text'] = df['text'].astype(str)

print("Rows:", len(df))
df.head(3)


Rows: 1500


user_name       bank_name bank_type platform            timestamp  \
0    Manish       Axis Bank   Private  Twitter  2024-09-02T13:54:18   
1   Keerthi  Bank of Baroda    Public  Twitter  2025-11-02T21:48:45   
2      Amit      ICICI Bank   Private  Twitter  2025-10-15T04:53:45   

                                                text   category  \
0  Axis Bank offered helpful service at the branc...    service   
1  Can't complete payment on Bank of Baroda. UPI ...    failure   
2  Claimed an offer from ICICI Bank and it worked...  promotion   

   sentiment_score sentiment  likes  retweets  user_followers  is_reply  \
0           -0.194  negative      1         0            7640     False   
1           -0.439  negative      6         0           15511      True   
2            0.702  positive      3         0           18699     False   

   in_reply_to_bank_response  response_time_minutes          location  
0                       True                   70.0      Delhi, India  
1                       True                    5.0  Hyderabad, India  
2                       True                  120.0    Kolkata, India

In [4]:
nlp = spacy.load("en_core_web_sm")


In [5]:
def clean_text(text):
    text = re.sub(r"http\S+|www\S+", " ", text)
    text = re.sub(r"@\w+|#\w+", " ", text)
    return re.sub(r"\s+", " ", text).strip()

df['text_clean'] = df['text'].apply(clean_text)
df[['text','text_clean']].head(3)


text  \
0  Axis Bank offered helpful service at the branc...   
1  Can't complete payment on Bank of Baroda. UPI ...   
2  Claimed an offer from ICICI Bank and it worked...   

                                          text_clean  
0  Axis Bank offered helpful service at the branc...  
1  Can't complete payment on Bank of Baroda. UPI ...  
2  Claimed an offer from ICICI Bank and it worked...

In [6]:
def get_ner_entities(text):
    doc = nlp(text)
    return [(ent.text, ent.label_) for ent in doc.ents]

df['entities_pretrained'] = df['text_clean'].apply(get_ner_entities)

df[['text_clean','entities_pretrained']].head(10)


text_clean  \
0  Axis Bank offered helpful service at the branc...   
1  Can't complete payment on Bank of Baroda. UPI ...   
2  Claimed an offer from ICICI Bank and it worked...   
3  Axis Bank customer service resolved my issue q...   
4        Loving the cashback offer on SBI app today!   
5  Transaction failed on HDFC Bank UPI — money de...   
6                UPI failed on SBI, very frustrating   
7        SBI promo gave instant discount, impressive   
8  HDFC Bank customer service resolved my issue q...   
9  Someone tried to scam me pretending to be SBI ...   

                      entities_pretrained  
0                      [(Axis Bank, ORG)]  
1    [(Bank of Baroda, ORG), (UPI, NORP)]  
2                     [(ICICI Bank, ORG)]  
3  [(Axis Bank, ORG), (Axis Mobile, ORG)]  
4             [(SBI, ORG), (today, DATE)]  
5         [(HDFC Bank, ORG), (UPI, NORP)]  
6               [(UPI, NORP), (SBI, ORG)]  
7                            [(SBI, ORG)]  
8                      [(HDFC Bank, ORG)]  
9                            [(SBI, ORG)]


NER PART 2 — Add Custom Entity Types (Banking-specific)

spaCy’s pretrained NER does NOT detect things like:

UPI

debit card

net banking

app names (Google Pay, PhonePe)

fraud keywords
We add them manually using PhraseMatcher.

In [7]:
matcher = PhraseMatcher(nlp.vocab, attr="LOWER")

custom_entities = {
    "BANK_SERVICE": ["upi", "net banking", "mobile banking", "atm", "debit card", "credit card"],
    "BANK_APP": ["gpay", "google pay", "phonepe", "paytm", "bhim"],
    "FRAUD_TERM": ["fraud", "scam", "phishing", "unauthorized", "hacked"],
    "PROMOTION": ["offer", "cashback", "reward", "discount"]
}

# Add patterns
for label, terms in custom_entities.items():
    patterns = [nlp.make_doc(t) for t in terms]
    matcher.add(label, patterns)


In [8]:
def get_custom_entities(doc):
    matches = matcher(doc)
    spans = []
    for match_id, start, end in matches:
        label = nlp.vocab.strings[match_id]
        span = Span(doc, start, end, label=label)
        spans.append(span)
    return spans


In [9]:
def extract_all_entities(text):
    doc = nlp(text)
    ents = list(doc.ents)                     # pretrained
    custom_ents = get_custom_entities(doc)    # custom
    return [(ent.text, ent.label_) for ent in ents + custom_ents]

df['entities_all'] = df['text_clean'].apply(extract_all_entities)

df[['text_clean','entities_all']].head(10)


text_clean  \
0  Axis Bank offered helpful service at the branc...   
1  Can't complete payment on Bank of Baroda. UPI ...   
2  Claimed an offer from ICICI Bank and it worked...   
3  Axis Bank customer service resolved my issue q...   
4        Loving the cashback offer on SBI app today!   
5  Transaction failed on HDFC Bank UPI — money de...   
6                UPI failed on SBI, very frustrating   
7        SBI promo gave instant discount, impressive   
8  HDFC Bank customer service resolved my issue q...   
9  Someone tried to scam me pretending to be SBI ...   

                                        entities_all  
0                                 [(Axis Bank, ORG)]  
1  [(Bank of Baroda, ORG), (UPI, NORP), (UPI, BAN...  
2            [(ICICI Bank, ORG), (offer, PROMOTION)]  
3             [(Axis Bank, ORG), (Axis Mobile, ORG)]  
4  [(SBI, ORG), (today, DATE), (cashback, PROMOTI...  
5  [(HDFC Bank, ORG), (UPI, NORP), (UPI, BANK_SER...  
6     [(UPI, NORP), (SBI, ORG), (UPI, BANK_SERVICE)]  
7                [(SBI, ORG), (discount, PROMOTION)]  
8                                 [(HDFC Bank, ORG)]  
9                   [(SBI, ORG), (scam, FRAUD_TERM)]

In [10]:
all_entities = []

for lst in df['entities_all']:
    for ent, label in lst:
        all_entities.append((ent, label))

ent_df = pd.DataFrame(all_entities, columns=['entity','label'])
ent_df.head()


entity         label
0       Axis Bank           ORG
1  Bank of Baroda           ORG
2             UPI          NORP
3             UPI  BANK_SERVICE
4      ICICI Bank           ORG

In [11]:
top_entities = ent_df['entity'].value_counts().head(20)
top_entities


entity
UPI               640
PNB               276
HDFC Bank         263
Axis Bank         225
SBI               218
Bank of Baroda    218
offer             112
Chatbot            80
Twitter            71
cashback           60
today              60
hours              58
discount           53
ICICI Bank         52
SBI/Banks:         46
phishing           35
YONO               31
fraud              30
scam               28
OTP                21
Name: count, dtype: int64

In [12]:
label_counts = ent_df['label'].value_counts()
label_counts


label
ORG             1598
BANK_SERVICE     343
PROMOTION        225
PERSON           151
NORP             124
FRAUD_TERM        93
DATE              60
TIME              58
Name: count, dtype: int64

In [13]:
fraud_samples = df[df['entities_all'].astype(str).str.contains("FRAUD_TERM")][['text_clean','entities_all']].head(10)
fraud_samples


text_clean  \
9    Someone tried to scam me pretending to be SBI ...   
27   Someone tried to scam me pretending to be PNB ...   
36   HDFC Bank app showing unknown beneficiary - po...   
49   Possible phishing attempt related to Axis Bank...   
67   PNB app showing unknown beneficiary - possible...   
87   Axis Bank app showing unknown beneficiary - po...   
112  SBI app showing unknown beneficiary - possible...   
115  Someone tried to scam me pretending to be PNB ...   
124  ICICI Bank app showing unknown beneficiary - p...   
134  PNB app showing unknown beneficiary - possible...   

                                          entities_all  
9                     [(SBI, ORG), (scam, FRAUD_TERM)]  
27                    [(PNB, ORG), (scam, FRAUD_TERM)]  
36             [(HDFC Bank, ORG), (fraud, FRAUD_TERM)]  
49   [(Axis Bank, ORG), (Axis Mobile, ORG), (phishi...  
67                   [(PNB, ORG), (fraud, FRAUD_TERM)]  
87             [(Axis Bank, ORG), (fraud, FRAUD_TERM)]  
112                  [(SBI, ORG), (fraud, FRAUD_TERM)]  
115                   [(PNB, ORG), (scam, FRAUD_TERM)]  
124                              [(fraud, FRAUD_TERM)]  
134                  [(PNB, ORG), (fraud, FRAUD_TERM)]